Übung: Kanal und Abonnenten mit K-native
-----------------------------

**Kanäle**
* Kanäle sind eine Ereignisweiterleitungs- und Persistenzschicht, wobei jeder Kanal eine separate benutzerdefinierte Kubernetes-Ressource ist. Ein Kanal kann von Apache Kafka oder InMemoryChannel unterstützt werden. Diese Übung verwendet InMemoryChannel.

**Abonnements**
* Mit Abonnements registrieren Sie Ihren Dienst, um einen bestimmten Kanal zu hören.

- - -

Zuerst erstellen wir den Kubernetes Namespace



In [ ]:
import os
os.environ['NS_CHNL']='ms-chnl'
! kubectl create namespace ${NS_CHNL}
! # kubectl label  namespace ${NS_CHNL} istio-injection=enabled

In [ ]:
%%bash
kubectl apply --namespace ${NS_CHNL} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-deployment.yaml
kubectl apply --namespace ${NS_CHNL} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-deployment.yaml
kubectl apply --namespace ${NS_CHNL} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-deployment.yaml
kubectl apply --namespace ${NS_CHNL} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-deployment.yaml 
kubectl apply --namespace ${NS_CHNL} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-service.yaml
kubectl apply --namespace ${NS_CHNL} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-service.yaml
kubectl apply --namespace ${NS_CHNL} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-service.yaml
kubectl apply --namespace ${NS_CHNL} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-service.yaml
kubectl get   --namespace ${NS_CHNL} pods,services

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ${NS_CHNL} webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

**Sales** starten wir im Serverless Modus und **Shipment** und **Invoicing** mit genau einer Instanz.

In [ ]:
%%bash
kubectl patch configmap/config-domain --namespace knative-serving --type merge --patch '{"data":{"dukmaster-10-default.mshome.net":""}}' 
#kn service create invoicing --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/backoffice/invoicing:4.0.0 --port 8080 --namespace ${NS_CHNL}
#kn service create shipment  --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/backoffice/shipment:4.0.0  --port 8080 --namespace ${NS_CHNL}
kn service create invoicing --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/cloudevents --port 8080 --namespace ${NS_CHNL}
kn service create shipment  --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/cloudevents --namespace ${NS_CHNL}

kn service create sales               --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/management/sales:3.2.0  --port 8080 --namespace ${NS_CHNL}

Um die Services via Browser anzusprechen ist im Windows die Datei `C:\Windows\System32\Drivers\etc\hosts` um folgende Einträge zu ergänzen.

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
echo $(cat ~/work/server-ip) shipment.${NS_CHNL}.dukmaster-10-default.mshome.net
echo $(cat ~/work/server-ip) invoicing.${NS_CHNL}.dukmaster-10-default.mshome.net
echo $(cat ~/work/server-ip) sales.${NS_CHNL}.dukmaster-10-default.mshome.net

echo ""
echo "Browser URLs"
# echo http://shipment.${NS_CHNL}.dukmaster-10-default.mshome.net:${PORT}/shipment
# echo http://invoicing.${NS_CHNL}.dukmaster-10-default.mshome.net:${PORT}/invoicing
echo http://shipment.${NS_CHNL}.dukmaster-10-default.mshome.net:${PORT}/
echo http://invoicing.${NS_CHNL}.dukmaster-10-default.mshome.net:${PORT}/

echo http://sales.${NS_CHNL}.dukmaster-10-default.mshome.net:${PORT}/sales

### Kanal (Senke)

Der Kanal oder die Senke ist eine Schnittstelle zwischen der Ereignisquelle und dem Abonnenten . 

Die Kanäle verarbeiten die eingehenden Ereignisse und verteilen die Ereignisdaten an die Abonnenten (Microservices).

Beim Weiterleiten von Ereignissen an Abonnenten transformiert der Kanal die Ereignisdaten gemäss der [CloudEvent- Spezifikation](http://cloudevents.io/).

- - -

Um die Microservices mit den Serverless Microservices **Shipment** und **Invoicing** zu verbinden, brauchen wir zuerst einen Channel und dann die entsprechenden Verbindungen vom Channel zu den Microservices.

In [ ]:
! kn channel create orders --namespace ${NS_CHNL}

### Ereignisabonnent

Das Ereignisabonnement  ist für die Verbindung des Kanals (Senke) mit dem Microservice (**Shipment** und **Invoicing**) verantwortlich. Sobald ein Microservice mit einem Kanal verbunden ist, beginnt er, die Ereignisse (Cloud-Ereignisse) zu empfangen.

In [ ]:
! kn channel list --namespace ${NS_CHNL}
! echo ""
! kn route list --namespace ${NS_CHNL}

In [ ]:
%%bash
kn subscription create shipment-sub  --channel orders --sink shipment  --namespace ${NS_CHNL}
kn subscription create invoicing-sub --channel orders --sink invoicing --namespace ${NS_CHNL}

### Gesamtübersicht

Die Subscriptions horchen auf den Channel `order` und leiten die Events weiter an die Microservices (**Shipment** und **Invoicing**) .

In [ ]:
%%bash
kn subscription list --namespace ${NS_CHNL}
echo ""
kn channel list --namespace ${NS_CHNL}
echo ""
kn service list --namespace ${NS_CHNL}

Nun müssen wir **Shipment** und **Invoicing** umbauen, damit sie REST POST Events empfangen können

In [ ]:
%%bash
kn source ping create orders-ping-source \
  --schedule '*/30 * * * * *' \
  --data '{"message": "Hello order"}' \
  --sink channel:orders \
  --namespace ${NS_CHNL} 
  

In [ ]:
! kn source list --namespace ${NS_CHNL} 

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
curl -X POST -H "Host: orders-kn-channel.ms-chnl.svc.cluster.local" http://localhost:${PORT} \
-H "Content-Type: application/json" -H "ce-id: orders" -d '{"message": "Hallo via POST local!"}'

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
curl -X POST -H "Host: orders-kn-channel.ms-chnl.dukmaster-10-default.mshome.net" -H "Content-Type: application/json" \
-H "ce-id: orders" -d '{"message": "Hallo, via POST dukmaster!"}' http://localhost:${PORT}    
    

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
curl -X POST -H "Host: imc-dispatcher.knative-eventing.svc.cluster.local" -H "Content-Type: application/json" \
-H "ce-id: orders" -d '{"message": "Hallo, via POST imc dukmaster!"}' http://localhost:${PORT}    

In [ ]:
! stern invoicing -c user-container --namespace ${NS_CHNL} --tail 200 

- - -
### Dashboard

Jetzt ist ein guter Zeitpunkt um das Kubernetes Dashboard zu starten und dort im Pulldownmenu den Namespace "ms-chnl" auszuwählen.

Wählt nachfolgenden Link an und aktzeptiert das Zertifikat um dann ohne Token, drückt "Überspringen" oder "Skip", ins Dashboard zu wechseln.

In [ ]:
! echo "https://"$(cat ~/work/server-ip)":18443"

---
## Kiali — Observability


Kiali vom, Projekt Istio, hilft die Struktur (= Service Mesh) der Applikation und deren Microservices und Schnittstellen zu verstehen.


In der Oberfläche wechselt rechts auf "Graph" und wählt als Namespace **ms-faas** aus.

Mittels Aktivierung der Option "Traffic Animation" im Pulldown "Display" wird die Kommunkation sichtbar

In [ ]:
! kubectl get service -n istio-system -l app=kiali  -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Kiali   UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app=kiali -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')

## Lasttest

Um die Verbindungen sichtbar zu machen, erzeugen wir ein wenig Traffic.

Dazu verwenden wir [hey](https://github.com/rakyll/hey) welche gleichzeitige parallele Request unterstützt.

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
hey -n 10000 -c 10 -H "Host: sales.${NS_CHNL}.dukmaster-10-default.mshome.net" http://localhost:${PORT}/sales

- - -

Aufräumen


In [ ]:
! # kubectl delete pod --all --namespace ${NS_CHNL} --grace-period=0 --force
! kn service delete shipment  --namespace ${NS_CHNL}
! kn service delete invoicing --namespace ${NS_CHNL}
! kn service delete sales --namespace ${NS_CHNL}
! kubectl delete namespace ${NS_CHNL}

In [ ]:
%%bash
kn source ping create shipment-ping-source \
  --schedule '*/2 * * * *' \
  --data '{"message": "Hello shipment"}' \
  --sink ksvc:shipment \
  --namespace ${NS_CHNL}  

In [ ]:
%%bash
kn source ping create invoicing-ping-source \
  --schedule "*/2 * * * *" \
  --data '{"message": "Hello invoicing"}' \
  --sink ksvc:invoicing \
  --namespace ${NS_CHNL}  
  